# Static Bertrand Game

This notebook runs two tournaments: one for both of the following demand functions: 

1. *perfect substitutes* (pure Bertrand), and
2.  *imperfect substitutes* (differentiated Bertrand).  

Your goal is to write a player function, `player.py` and put it in a subfolder, `submission`. 

To test your player function and experiment, you can write multiple functions, called `player1.py`, `player2.py`,..., and put them all in `/players/`. When you run the tournament (in this notebook), it will automatically load them all and run an all-play-all tournament and declare the winner. 

Note: you should write your player function so that it performs well under both types of demand function. You can easily check which you are facing by investigating the magnitude of the change in profit when you lower price by a tiny amount, e.g. `1e-8`. 

**Short version:** This notebook is the short version, and there is an accompanying notebook, which is longer and has more details.

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

%load_ext autoreload
%autoreload 2 
from game_tournament.game import StaticBertrandGame, Tournament

In [2]:
from scipy.optimize import minimize_scalar 

In [3]:
# later, we will give the path directly to a function
player_path = './players/'

# General setup, common across specifications 

* **Marginal cost:** $c = 0.5$ 
* **Action space:** $p \in [c; 6]$

In [4]:
c = 0.5
pmin = c
pmax = 6.

# Part 1: Imperfect substitutes (differentiated Bertrand)
$$ D_i(p_i, p_j) = \frac{\exp(\beta_0 - \beta_1 p_i)}{1 + \exp(\beta_0-\beta_1p_i) + \exp(\beta_0-\beta_1 p_j)} $$

In [5]:
def demand(p1, p2): 
    '''
    Args
        p1,p2: prices 
    Returns: 
        demand to firm 1
    '''
    b0 = 5.
    b1 = 2.
    u1 = np.exp(b0-b1*p1)
    u2 = np.exp(b0-b1*p2)
    u0 = 1.0 
    
    denom = u0 + u1 + u2
    
    # market shares 
    s1 = u1/denom 
    s2 = u2/denom
    
    return s1,s2

def profit1(p1, p2):
    # c is read in as a global 
    s1,s2 = demand(p1,p2)
    return s1*(p1-c)

def profit2(p2, p1): # note the order of inputs! 
    s1,s2 = demand(p1,p2)
    return s2*(p2-c)

# Running a Tournament

This will run a full tournament between all the player functions found in the folder `player_path`. 

In [6]:
T = 100
game_data = {'profit_function1':profit1, 'profit_function2':profit2, 'price_range': (pmin,pmax)
}
t = Tournament(player_path, StaticBertrandGame, game_data=game_data, T=T, tournament_name='Total', )
t.run() # run the tournament and score the game 

3it [00:01,  2.70it/s]

Tournament winner was: BR to midpoint (against 2 opponents)


,Total
Player,
BR to midpoint,0.731441
First-order,0.723072
0th order,0.230327


In [7]:
matchup_results = t.get_matchup_results()
matchup_results

Opponent,0th order,BR to midpoint,First-order
Player,,,
0th order,NaN,0.228489,0.232166
BR to midpoint,0.831749,NaN,0.631132
First-order,0.831790,0.614353,NaN


# Part 2: Near-perfect Substitutes
$$ D_i(p_i, p_j) = \frac{\exp(\beta_0 - \beta_1 p_i + \beta_2 R(p_i,p_j))}{1 + \exp(\beta_0-\beta_1p_i + \beta_2 R(p_i,p_j)) + \exp(\beta_0-\beta_1 p_j + \beta_2 R(p_j,p_i)) } $$

where the Recommendation-effect, $R$, is an attraction to the cheapest product: 

$$ 
R(p_i,p_j) = 
\begin{cases}
1 & \text{if } p_i < p_j \\
\frac{1}{2} & \text{if } p_i = p_j \\
0 & \text{if } p_i > p_j.
\end{cases}
$$

Note that $R$ is a discontinuous function, which results in $D(p_i, p_j)$ being discontinuous. 

In [8]:
def demand(p1, p2): 
    '''
    Args
        p1,p2: prices 
    Returns: 
        demand to firm 1
    '''
    b0 = 5.
    b1 = 2.
    b2 = 2.

    # price recommendations: bonus for being the cheapest firm (shared in case of ties)
    R1 = 1.*(p1<p2) + 0.5*(p1==p2)
    R2 = 1.*(p2<p1) + 0.5*(p1==p2)

    # utility for each product 
    u1 = np.exp(b0-b1*p1+b2*R1)
    u2 = np.exp(b0-b1*p2+b2*R2)
    u0 = 1.0 
    
    # denominator in the logit choice probability 
    denom = u0 + u1 + u2
    
    # market shares 
    s1 = u1/denom 
    s2 = u2/denom
    
    return s1,s2

def profit1(p1, p2):
    # c is read in as a global 
    s1,s2 = demand(p1,p2)
    return s1*(p1-c)

def profit2(p2, p1): # note the order of inputs! 
    s1,s2 = demand(p1,p2)
    return s2*(p2-c)

## Tournament

In [9]:
T = 100 # just to smooth out randomness
game_data = {'profit_function1':profit1, 'profit_function2':profit2, 'price_range': (pmin,pmax)
}
t = Tournament(player_path, StaticBertrandGame, game_data=game_data, T=T, tournament_name='Total', )
t.run() # run the tournament and score the game 

3it [00:07,  2.49s/it]

Tournament winner was: BR to midpoint (against 2 opponents)


,Total
Player,
BR to midpoint,1.332734
First-order,0.720479
0th order,0.406679


In [10]:
t.get_matchup_results()

Opponent,0th order,BR to midpoint,First-order
Player,,,
0th order,NaN,0.445126,0.368232
BR to midpoint,1.023986,NaN,1.641482
First-order,1.225518,0.215441,NaN
